In [56]:
using JSON, DelimitedFiles
clmp = vec(readdlm("./data/cleaned_ercot2019_RTprices.csv", ',', Float64, '\n'));
clmp = abs.(clmp) / 1e3;  # problem is unbounded with negative prices, convert from $/MWh to $/kWh

In [2]:
"""
    get_dict(fp::String)

load in a JSON file from `fp` and return a Dict
"""
function get_dict(fp::String)
    JSON.parsefile(fp)
end

get_dict

In [91]:
r = get_dict("results_pwf1x.json")

Dict{String, Any} with 2768182 entries:
  "vsqrd[26,2789]"            => 0.809491
  "yi[31,7031]"               => 0.0304278
  "Qⱼ[22,8046]"               => -0.00169031
  "bydd[9,3236]"              => 0.0
  "Qⱼ[8,7994]"                => 0.0
  "Qⱼ[25,5164]"               => -0.022689
  "bye[17,6515]"              => 1.0
  "Qᵢⱼ[11-12,7360]"           => 1.184
  "bye[34,2242]"              => 1.0
  "Qⱼ[8,6856]"                => 0.0
  "ytemperature[5,2932]"      => -1.44192
  "xe[9,4931]"                => 0.25
  "lambda_warehouse[10,5279]" => 106.413
  "byi[9,2796]"               => 1.0
  "Pᵢⱼ[0-1,3153]"             => 34.5458
  "Qⱼ[5,636]"                 => -0.305425
  "Qⱼ[0,8501]"                => 1.0113
  "Pⱼ[9,4121]"                => -0.0137877
  "vsqrd[34,11]"              => 1.00445
  "Qⱼ[18,3455]"               => -0.00191262
  "xe[34,4770]"               => 0.25
  "bypvprod[9,1587]"          => 0.0
  "yi[15,4201]"               => 10.7912
  "Pⱼ[12,3912]"               => -0

In [86]:
"""
    get_vals(v::String, d::Dict)

Given the variable name `v` find all values in Dict `d` 
that contain `v` in the keys
"""
function get_vals(s::String, d::Dict; T=8760)
    filterkeys = filter(n -> startswith(n, s*"["), keys(r))
    if T > 1
        try
            nnodes = Int(length(filterkeys) / T)
            if nnodes > 1
                a = Dict{String, Vector{Float64}}()
                for k in filterkeys
                    n = k[ findfirst('[', k)+1: findfirst(',', k)-1]
                    t = parse(Int, k[ findfirst(',', k)+1: findfirst(']', k)-1])
                    if !(n in keys(a))
                        a[n] = zeros(T)
                    end
                    a[n][t] = d[k]
                end
                return a
            else  # for no node indices
                a = zeros(T)
                if !(length(filterkeys) == T)
                    @warn "There are $(length(filterkeys)) filterkeys, and looking for $T values!"
                end
                for k in filterkeys
                    t = parse(Int, k[ findfirst('[', k)+1: findfirst(']', k)-1])
                    a[t] = d[k]
                end
                return a  
            end
        catch
            println(filterkeys)
        end
    elseif T == 0
        try
            nnodes = Int(length(filterkeys))
            a = Dict{String, Float64}()
            for k in filterkeys
                n = k[ findfirst('[', k)+1: findfirst(']', k)-1]
                a[n] = d[k]
            end
            return a
        catch
            println("error: ", filterkeys)
        end
        
    end
end

get_vals

In [65]:
xi = get_vals("xi", r)

Dict{String, Vector{Float64}} with 3 entries:
  "15" => [318.543, 313.228, 307.692, 320.523, 92.1232, 309.842, 322.762, 94.45…
  "10" => [94.83, 89.493, 83.9336, 96.74, 91.4825, 86.006, 98.8987, 93.7312, 88…
  "5"  => [315.983, 310.562, 304.915, 317.63, 89.1099, 306.703, 319.493, 91.049…

In [66]:
pv = get_vals("ypv", r, T=0)

Dict{String, Float64} with 5 entries:
  "17" => 0.0435739
  "31" => 0.0
  "34" => 0.890921
  "22" => 0.122285
  "9"  => 0.970694

In [39]:
Pij = get_vals("Pᵢⱼ", r)

Dict{String, Vector{Float64}} with 37 entries:
  "21-22" => [0.136087, 0.133532, 0.113894, 0.113031, 0.119028, 0.131037, 0.196…
  "13-14" => [0.176791, 0.162161, 0.142497, 0.141604, 0.69702, 0.171869, 0.2688…
  "0-1"   => [0.778784, 0.761265, 0.749562, 0.749169, 2.42774, 0.822769, 1.0813…
  "15-16" => [0.159246, 0.150979, 0.131326, 0.130444, 0.136623, 0.154279, 0.239…
  "7-26"  => [0.0108485, 0.0108485, 0.0108485, 0.0108485, 0.0109213, 0.0109764,…
  "4-5"   => [0.745814, 0.72038, 0.705068, 0.703469, 2.38055, 0.774709, 1.03195…
  "16-17" => [0.159246, 0.150979, 0.131326, 0.130444, 0.136623, 0.154279, 0.239…
  "5-6"   => [0.732606, 0.707173, 0.691861, 0.690262, 1.81815, 0.761454, 1.0185…
  "11-12" => [0.280519, 0.265814, 0.260379, 0.260049, 0.821037, 0.291281, 0.392…
  "20-21" => [0.136087, 0.133532, 0.113894, 0.113031, 0.119028, 0.131037, 0.196…
  "7-29"  => [0.243068, 0.239053, 0.253881, 0.25419, 0.263931, 0.268764, 0.3305…
  "6-7"   => [0.692524, 0.668367, 0.658969, 0.657646, 1.78451,

In [40]:
bkw = get_vals("xbkW", r; T=0)

Dict{String, Float64} with 3 entries:
  "24" => 0.0
  "2"  => 0.0
  "7"  => 0.0

In [41]:
bkwh = get_vals("xbkWh", r; T=0)

Dict{String, Float64} with 3 entries:
  "24" => 0.0
  "2"  => 0.0
  "7"  => 0.0

In [33]:
ye = get_vals("ye", r)

Dict{String, Vector{Float64}} with 5 entries:
  "17" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.149313, 0.27345  …  0.3728…
  "31" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5862, 2.92152  …  0.649822…
  "34" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.4532, 6.41499  …  8.71462,…
  "22" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.557994, 1.03383  …  0.0, 0…
  "9"  => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.49669, 10.2489  …  14.0913…

In [34]:
ypvprod = get_vals("ypvprod", r)

Dict{String, Vector{Float64}} with 5 entries:
  "17" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.150885, 0.274954  …  0.374…
  "31" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.35073, 2.4614  …  0.0, 2.6…
  "34" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.58262, 6.52853  …  8.88856…
  "22" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.576077, 1.04977  …  0.0137…
  "9"  => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.72784, 10.4377  …  14.2109…

In [42]:
results_list = [
    "results_pwf1x.json", 
    "results_pwf2x.json", 
    "results_pwf3x.json", 
    "results_pwf4x.json", 
    "results_pwf5x.json"
]

5-element Vector{String}:
 "results_pwf1x.json"
 "results_pwf2x.json"
 "results_pwf3x.json"
 "results_pwf4x.json"
 "results_pwf5x.json"

In [43]:
rdicts = Dict[r]
for rstr in results_list[2:end]
    push!(rdicts, get_dict(rstr))
end

# capacity results

In [58]:
pvs = Dict[]
bkWs = Dict[]
bkWhs = Dict[]
for rd in rdicts
    push!(pvs, get_vals("ypv", rd, T=0))
    push!(bkWs, get_vals("xbkW", rd, T=0))
    push!(bkWhs, get_vals("xbkWh", rd, T=0))
end

In [59]:
for pv in pvs
    println([round(p, digits=3) for p in values(pv)])
end

[0.044, 0.0, 0.891, 0.122, 0.971]
[0.064, 0.0, 1.419, 0.189, 1.66]
[0.143, 0.419, 2.841, 0.402, 3.499]
[0.348, 2.664, 8.235, 1.532, 12.845]
[0.559, 5.004, 13.273, 2.134, 21.22]


In [60]:
for b in bkWs
    println([round(p, digits=3) for p in values(b)])
end

[0.0, 0.0, 0.0]
[0.274, 0.288, 0.314]
[0.768, 0.179, 0.0]
[0.295, 0.351, 0.0]
[0.342, 0.457, 0.275]


In [62]:
for b in bkWhs
    println([round(p, digits=3) for p in values(b)])
end

[0.0, 0.0, 0.0]
[0.577, 0.607, 0.662]
[1.473, 0.341, 0.0]
[0.621, 0.739, 0.0]
[0.72, 0.962, 0.58]


# cost results

In [51]:
LLnodes_withPV = ["9", "22", "31", "34", "17"]
LLnodes_warehouse = ["5", "10", "15"]

xis = Dict[]
yis = Dict[]
xes = Dict[]
yes = Dict[]
for rd in rdicts
    push!(xis, get_vals("xi", rd, T=8760))
    push!(yis, get_vals("yi", rd, T=8760))
    push!(xes, get_vals("xe", rd, T=8760))
    push!(yes, get_vals("ye", rd, T=8760))
end

In [53]:
warehouse_cost = zeros(5)

for i = 1:5
    for n in LLnodes_warehouse
       warehouse_cost[i] += sum(
            yis[i][n] .* xis[i][n]
        )
    end
end

In [110]:
maximum(values(xis[5]))

8760-element Vector{Float64}:
 317.3391962752986
 311.97472250438346
 306.38672899301355
 319.16325936772245
  90.70650483702758
 315.7804474184536
 328.94838272755584
 100.89934167018737
  95.81521909858556
 109.11661503555293
 104.37487885417468
  99.43557033190568
 112.88781423692802
   ⋮
  62.63004492717782
 123.59669964479748
  68.36666035221313
 110.97498376349003
  55.2190396425179
 271.3998545417273
 264.1212415319133
 256.5393529800238
 248.64155240513892
 240.41467680629952
 231.8450147241761
   0.0

In [54]:
warehouse_cost

5-element Vector{Float64}:
 1.4526749800539404e7
 1.4064967667953836e7
 1.3868775260688901e7
 1.3483482871665575e7
 1.428228833847922e7

In [55]:
pv_benefit = zeros(5)

for i = 1:5
    for n in LLnodes_withPV
       pv_benefit[i] += sum(
            yes[i][n] .* xes[i][n]
        )
    end
end
pv_benefit

5-element Vector{Float64}:
   330.3763110224797
   771.9338445737885
  2052.7800582555824
  8016.518892143267
 13288.78475772311

In [88]:
x0s = Vector[]
for rd in rdicts
    push!(x0s, get_vals("x0", rd, T=8760))
end

In [89]:
whl_cost = zeros(5)
# NOTE not accounting for pwf and its scaler

for i = 1:5
    whl_cost[i] += sum(
        clmp .* x0s[i]
    )
end
whl_cost

5-element Vector{Float64}:
 2520.881759154236
 2388.944373344213
 2236.2580754796327
 1674.757982979163
 1247.2198365359654

In [94]:
ULnodes_withBESS = ["2", "7", "24"]
bplusses = zeros(5)

for i = 1:5
    bplusdict = get_vals("xbplus", rdicts[i], T=8760)
    for n in ULnodes_withBESS
        bplusses[i] += sum(bplusdict[n])
    end
end

In [95]:
bplusses

5-element Vector{Float64}:
    0.0
 1026.4807730426182
 1020.5853364644552
  752.0345354121262
 1313.5196262529098

In [102]:
whl_energy_purchased = zeros(5)
pv_energy_purchased = zeros(5)

for i = 1:5
    whl_energy_purchased[i] += sum(x0s[i])
    pv_energy_purchased[i] += sum(sum(values(yes[i])))
end

In [103]:
whl_energy_purchased

5-element Vector{Float64}:
 129895.39437256625
 127952.535597669
 121799.65220969915
  94092.88530047641
  72352.46377115446

In [104]:
pv_energy_purchased

5-element Vector{Float64}:
  2396.557296924839
  4162.787431130075
  9890.978717250542
 36981.995971833785
 61378.87145780142

PV and cost trends make sense, but not sure about the battery trends. maybe due to voltage limits and excess PV?

LinDistFlow is valid with back flow?

In [128]:
maxvs = zeros(5)
for i = 1:5
    vdict = get_vals("vsqrd", rdicts[i], T=8760)
    maxvs[i] = maximum([maximum(sqrt.(v)) for v in values(vdict)])
end

In [129]:
maxvs

5-element Vector{Float64}:
 1.0034177834335292
 1.0054777847509568
 1.011352705915531
 1.0332572710462962
 1.05

# plot day with peak price

In [111]:
using Plots

LoadError: ArgumentError: Package Plots not found in current path:
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.
